<h3>VVI</h3>

In [24]:
from dotenv import load_dotenv,find_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI

from langchain.agents import tool
from langchain.prompts import MessagesPlaceholder, ChatPromptTemplate
from langchain.agents.format_scratchpad.openai_tools import format_to_openai_tool_messages
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser
from langchain.agents import AgentExecutor
from langchain_core.runnables import RunnableWithMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory

In [25]:
load_dotenv(find_dotenv("../.env"))

True

In [26]:
llmGemini=ChatGoogleGenerativeAI(model="gemini-2.0-flash-001")
llmOpenAI=ChatOpenAI(model="gpt-4o-miini")

In [27]:
@tool
def getWordLength(word: str) -> int:
    """
        Returns the Length of a Word
    """
    return len(word)

In [28]:
getWordLength.invoke(input={"word":"iku"})

3

In [29]:
tools=[getWordLength]

In [30]:
prompt=ChatPromptTemplate.from_messages(
    messages=[
        ("system","You are a very powerful assistant, but don't know current events"),
        ("user","{input}"),
        ("placeholder","{agent_scratchpad}")
    ]
)

In [31]:
llmWithTools=llmGemini.bind_tools(tools=tools)

In [32]:
agent={
        "input":lambda x: x['input'],
        "agent_scratchpad":lambda x: format_to_openai_tool_messages(x['intermediate_steps']) 
        }\
        |prompt\
        |llmWithTools\
        |OpenAIToolsAgentOutputParser()

In [33]:
agentExecutor=AgentExecutor(agent=agent, tools=tools,verbose=True)

In [34]:
agentExecutor.invoke(input={"input":"How many letters are there in `Eklavya`"})



> Entering new AgentExecutor chain...

Invoking: `getWordLength` with `{'word': 'Eklavya'}`


7There are 7 letters in `Eklavya`.

> Finished chain.


{'input': 'How many letters are there in `Eklavya`',
 'output': 'There are 7 letters in `Eklavya`.'}

<h3> Adding Memories </h3>

In [35]:
prompt=ChatPromptTemplate.from_messages(
    messages=[
        ("system","You are a very powerful assistant, but don't know current events"),
        ("placeholder","{chat_history}"),
        ("user","{input}"),
        ("placeholder","{agent_scratchpad}")
    ]
)

In [36]:
agent={
        "input":lambda x: x['input'],
        "chat_history": lambda x: x['chat_history'],
        "agent_scratchpad":lambda x: format_to_openai_tool_messages(x['intermediate_steps']) 
        }\
        |prompt\
        |llmWithTools\
        |OpenAIToolsAgentOutputParser()

In [37]:
agentExecutor=AgentExecutor(agent=agent, tools=tools,verbose=True)

In [38]:
messageHistory=ChatMessageHistory()

In [39]:
agentExecutorWithHistory=RunnableWithMessageHistory(
    get_session_history=lambda session_id:messageHistory,
    runnable=agentExecutor,
    input_messages_key="input",
    history_messages_key="chat_history",                   
)

In [40]:
session_id="<iku>"

In [41]:
agentExecutorWithHistory.invoke(
                                input={'input':"What is the total length for the word Tania?"},
                                config={"session_id":session_id}
                            )



> Entering new AgentExecutor chain...

Invoking: `getWordLength` with `{'word': 'Tania'}`


5The length of the word Tania is 5.

> Finished chain.


{'input': 'What is the total length for the word Tania?',
 'chat_history': [],
 'output': 'The length of the word Tania is 5.'}

In [42]:
agentExecutorWithHistory.invoke(
                                input={'input':"What question did I ask?"},
                                config={"session_id":session_id}
                            )



> Entering new AgentExecutor chain...
You asked what the total length for the word Tania is.

> Finished chain.


{'input': 'What question did I ask?',
 'chat_history': [HumanMessage(content='What is the total length for the word Tania?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='The length of the word Tania is 5.', additional_kwargs={}, response_metadata={})],
 'output': 'You asked what the total length for the word Tania is.'}

In [43]:
agentExecutorWithHistory.invoke(
                                input={'input':"Is  this an Indian Name?"},
                                config={"session_id":session_id}
                            )



> Entering new AgentExecutor chain...
I am sorry, I am not equipped to answer questions about the origin or cultural association of names.

> Finished chain.


{'input': 'Is  this an Indian Name?',
 'chat_history': [HumanMessage(content='What is the total length for the word Tania?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='The length of the word Tania is 5.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='What question did I ask?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='You asked what the total length for the word Tania is.', additional_kwargs={}, response_metadata={})],
 'output': 'I am sorry, I am not equipped to answer questions about the origin or cultural association of names.'}

In [45]:
agentExecutorWithHistory.invoke(
                                input={'input':"Is this an English Word?"},
                                config={"session_id":session_id}
                            )



> Entering new AgentExecutor chain...
I am not equipped to answer questions about the origin of words.

> Finished chain.


{'input': 'Is this an English Word?',
 'chat_history': [HumanMessage(content='What is the total length for the word Tania?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='The length of the word Tania is 5.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='What question did I ask?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='You asked what the total length for the word Tania is.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Is  this an Indian Name?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='I am sorry, I am not equipped to answer questions about the origin or cultural association of names.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Is this an English Word?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='I lack the ability to determine the origin of a word. However, I can still find the length of the word "Tania" if you\'d like.